In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from classy import Class
from colossus.cosmology import cosmology as ccosmology
from scipy.interpolate import UnivariateSpline

from SSLimPy.cosmology import nonlinear
from SSLimPy.interface import sslimpy
from SSLimPy.utils.utils import linear_interpolate

plt.style.use("fivethirtyeight")

In [ ]:
# colosus results
ccosmo = ccosmology.setCosmology("planck18")

In [ ]:
# SSLimPy results
cosmodict = {
    "h": ccosmo.h,
    "Omegam": ccosmo.Om0,
    "Omegab": ccosmo.Ob0,
    "ns": ccosmo.ns,
    "sigma8": ccosmo.sigma8,
}

A = sslimpy.sslimpy(cosmopars=cosmodict)
nl = nonlinear.nonlinear("clustering", cosmology=A.curent_cosmo)

In [ ]:
k = nl.k
Dk = 4 * np.pi * (k / 2 / np.pi) ** 3 * nl.matpow(k, 0, tracer="clustering")
t = np.linspace(0, 1, 400)
rR = np.geomspace(1e-4, 1e3, 8)
I = np.empty((*t.shape, *rR.shape))
for ir, Ri in enumerate(rR):
    I = nonlinear.sigma_integrand(t, Ri, k, Dk, 3)
    plt.semilogy(t, I, label="R={:1.0e}".format(Ri))
plt.legend(loc=8, ncol=4)

In [ ]:
cosmodict = {
    "h": ccosmo.h,
    "Omega_m": ccosmo.Om0,
    "Omega_b": ccosmo.Ob0,
    "n_s": ccosmo.ns,
    "sigma8": ccosmo.sigma8,
    "N_ncdm": 1,
    "m_ncdm": 0.06,
    "N_ur": 2.0308,
}

cosmo = Class()
cosmo.set_default()
cosmo.set({"non linear": "hmcode2020"})
cosmo.set(cosmodict)
cosmo.compute()
class_sigma = np.zeros(len(nl.R))
for iR, Ri in enumerate(nl.R.value):
    class_sigma[iR] = cosmo.sigma_cb(Ri, 0)

In [ ]:
plt.loglog(nl.R, nl.sigmaR_lut[:, 0], label="New integration method")
plt.loglog(nl.R, class_sigma, label="Class")
plt.loglog(nl.R, nl.sigmaR_of_z(nl.R, 0), label="Old integration method")
Rh = np.geomspace(1e-4, 1e3)
plt.loglog(Rh / ccosmo.h, ccosmo.sigma(Rh, 0), label="Colosus", ls="--")
plt.xlabel(r"$R \:[\mathrm{Mpc}]$")
plt.ylabel(r"$\sigma(R)$")
plt.legend()